In [11]:
# NLP_mvp_refactored.py

# ## NLP: Análisis de Sentimiento Robusto

# #### 1. Importación de Librerías
# Se importan todas las librerías necesarias al inicio para una mejor gestión de dependencias.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
#from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import os


In [12]:

# Descargar e instalar el modelo de spaCy si no está presente
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print('Descargando el modelo en_core_web_sm de spaCy...')
    # La siguiente línea es para ejecutar en un entorno de notebook/terminal
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')



In [13]:
import random

# #### 2. Carga y Exploración de Datos (con muestra aleatoria)
# Leemos una muestra aleatoria para acelerar el prototipado sin sesgos.

try:
    # --- PARÁMETROS PARA EL MUESTREO ---
    total_filas = 1600000
    filas_a_leer = 20000
    probabilidad = filas_a_leer / total_filas

    # --- LÍNEA MODIFICADA ---
    # Usamos skiprows con una función lambda para muestreo aleatorio
    df = pd.read_csv('data/training.1600000.processed.noemoticon.csv',
        encoding='latin-1',
        header=None,
        # Omitir una fila si un número aleatorio es mayor que nuestra probabilidad
        skiprows=lambda i: i > 0 and random.random() > probabilidad
    )
    print(f'Dataset cargado exitosamente (muestra aleatoria de ~{filas_a_leer} filas).')
    print(df.head())

    # Asignar nombres a las columnas
    df.columns = ['Sentiment', 'id', 'date', 'query', 'user', 'Text']
    
    # Eliminar columnas innecesarias
    cols_to_drop = ['id', 'date', 'query']
    df = df.drop(columns=cols_to_drop)
    
    print('\nPrimeras 10 filas de la muestra aleatoria:')
    print(df.head(10))
    
    print(f'\nDimensiones de la muestra: {df.shape}')

except FileNotFoundError:
    print('Error: El archivo no fue encontrado.')
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Dataset cargado exitosamente (muestra aleatoria de ~20000 filas).
   0           1                             2         3                4  \
0  0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1  0  1467822687  Mon Apr 06 22:22:52 PDT 2009  NO_QUERY    xVivaLaJuicyx   
2  0  1467837762  Mon Apr 06 22:26:48 PDT 2009  NO_QUERY          Dogbook   
3  0  1467842607  Mon Apr 06 22:28:09 PDT 2009  NO_QUERY  VanessaSingline   
4  0  1467844505  Mon Apr 06 22:28:38 PDT 2009  NO_QUERY       luimoral85   

                                                   5  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  @BatManYNG I miss my ps3, it's out of commissi...  
2  Emily will be glad when Mommy is done training...  
3  @BridgetsBeaches Thank you for letting people ...  
4              I don't understand... I really don't   

Primeras 10 filas de la muestra aleatoria:
   Sentiment             user  \
0          0  _TheSpecialOne_   
1          0    xVivaLaJuicyx  

In [14]:
# #### 3. Mapeo de Sentimientos (Simplificado)
# Mapeamos los valores numéricos del sentimiento (0 y 4) a etiquetas de texto.
if 'df' in locals():
    # El valor 0 es Negativo y 4 es Positivo.
    df['Sentiment'] = df['Sentiment'].map({0: 'Negative', 4: 'Positive'})
    
    # Creamos la copia después del mapeo
    df_copy = df.copy()

    print('Distribución de sentimientos tras el mapeo:')
    print(df['Sentiment'].value_counts())

    

Distribución de sentimientos tras el mapeo:
Sentiment
Positive    9917
Negative    9849
Name: count, dtype: int64


In [15]:

# #### 4. Preprocesamiento de Texto
# Función de limpieza robusta que maneja posibles valores no textuales, convierte a minúsculas, elimina caracteres no deseados, lematiza y quita *stop words*.
def limpieza(texto):
    """Realiza una limpieza completa del texto, incluyendo manejo de casos nulos."""
    if not isinstance(texto, str):
        return ""
    
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    # Procesar con spaCy solo si el texto no está vacío
    if not texto:
        return ""
        
    doc = nlp(texto)
    texto_limpio = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    
    return texto_limpio

# ### Preparación de Datos para Modelado
if 'df_copy' in locals():
    X = df_copy['Text'].apply(limpieza)
    
    # Mapeamos 'Negative' a -1 y 'Positive' a 1
    y = df_copy['Sentiment'].map({'Negative': -1, 'Positive': 1})

    # Eliminar filas donde 'y' podría ser NaN si hubo valores de sentimiento inesperados
    # y asegurarse de que los índices coincidan
    valid_indices = y.notna()
    X = X[valid_indices]
    y = y[valid_indices]

    # Dividir los datos, estratificando por 'y' para mantener la proporción de clases
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    print(f"\n- Data para modelado preparada -")
    print(f"Tamaño de X_train: {X_train.shape[0]}")
    print(f"Tamaño de X_test: {X_test.shape[0]}")
    print(f"\nDistribución en y_train:\n{y_train.value_counts()}")


- Data para modelado preparada -
Tamaño de X_train: 13836
Tamaño de X_test: 5930

Distribución en y_train:
Sentiment
 1    6942
-1    6894
Name: count, dtype: int64


In [22]:

# ### Modelo 3: TF-IDF + Regresión Logística
# Este método pondera la importancia de las palabras según su frecuencia en el documento y en todo el corpus (TF-IDF). Se combina con una Regresión Logística, un modelo lineal robusto para clasificación.
if 'X_train' in locals():
    # Vectorización con TF-IDF
    vectorizer_tfidf = TfidfVectorizer()
    X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
    X_test_tfidf = vectorizer_tfidf.transform(X_test)

    # Entrenamiento del modelo de Regresión Logística
    # Se aumenta max_iter para asegurar la convergencia
    model_lr = LogisticRegression(random_state=42, max_iter=1000)
    model_lr.fit(X_train_tfidf, y_train)

    # Predicciones
    y_train_pred_lr = model_lr.predict(X_train_tfidf)
    y_test_pred_lr = model_lr.predict(X_test_tfidf)

    # Reporte de Clasificación
    print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---")
    print(classification_report(y_train, y_train_pred_lr))
    print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---")
    print(classification_report(y_test, y_test_pred_lr))


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---
              precision    recall  f1-score   support

          -1       0.87      0.85      0.86      6894
           1       0.85      0.87      0.86      6942

    accuracy                           0.86     13836
   macro avg       0.86      0.86      0.86     13836
weighted avg       0.86      0.86      0.86     13836


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---
              precision    recall  f1-score   support

          -1       0.74      0.71      0.72      2955
           1       0.72      0.75      0.73      2975

    accuracy                           0.73      5930
   macro avg       0.73      0.73      0.73      5930
weighted avg       0.73      0.73      0.73      5930



In [29]:
# Vectorización TF-IDF Optimizada
vectorizer_tfidf_opt = TfidfVectorizer(
    ngram_range=(1, 2),  # Considera palabras individuales y pares de palabras (bigramas)
    max_features=20000,  # Limita el vocabulario a las 20,000 características más importantes
    min_df=5,            # La palabra debe aparecer en al menos 5 documentos
    max_df=0.7           # La palabra no debe aparecer en más del 70% de los documentos
)

# Luego usas este nuevo vectorizador para transformar tus datos
X_train_tfidf_opt = vectorizer_tfidf_opt.fit_transform(X_train)
X_test_tfidf_opt = vectorizer_tfidf_opt.transform(X_test)

# Entrenamiento del modelo de Regresión Logística
# Se aumenta max_iter para asegurar la convergencia
model_lr = LogisticRegression(random_state=42, max_iter=1000)
model_lr.fit(X_train_tfidf_opt, y_train)

# Predicciones
y_train_pred_lr = model_lr.predict(X_train_tfidf_opt)
y_test_pred_lr = model_lr.predict(X_test_tfidf_opt)

# Reporte de Clasificación
print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---")
print(classification_report(y_train, y_train_pred_lr))
print("\n--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---")
print(classification_report(y_test, y_test_pred_lr))



--- Reporte de Clasificación: TF-IDF + Regresión Logística (Train) ---
              precision    recall  f1-score   support

          -1       0.81      0.78      0.79      6894
           1       0.79      0.82      0.80      6942

    accuracy                           0.80     13836
   macro avg       0.80      0.80      0.80     13836
weighted avg       0.80      0.80      0.80     13836


--- Reporte de Clasificación: TF-IDF + Regresión Logística (Test) ---
              precision    recall  f1-score   support

          -1       0.74      0.70      0.72      2955
           1       0.72      0.75      0.74      2975

    accuracy                           0.73      5930
   macro avg       0.73      0.73      0.73      5930
weighted avg       0.73      0.73      0.73      5930



Guardar los archivos

Cargar los archivos para usarlos

In [31]:


# 1. Cargar los objetos guardados
vectorizer_cargado = joblib.load('model/schimizzi_vectorizer.joblib')
modelo_cargado = joblib.load('model/schimizzi_modelo_s.joblib')

# 2. Nuevos datos para predecir
nuevos_datos = [
    "I love this product, it's the best thing I've ever bought!", # Positivo
    "This is a terrible experience, I'm so disappointed.",     # Negativo
]

# 3. Usar el VECTORIZER CARGADO para transformar el texto
nuevos_datos_transformados = vectorizer_cargado.transform(nuevos_datos)

# 4. Usar el MODELO CARGADO para hacer la predicción
predicciones = modelo_cargado.predict(nuevos_datos_transformados)

# 5. Interpretar el resultado (-1: Negativo, 1: Positivo)
for texto, sentimiento in zip(nuevos_datos, predicciones):
    resultado = "Positivo" if sentimiento == 1 else "Negativo"
    print(f"Texto: '{texto}' -> Sentimiento Predicho: {resultado}")

Texto: 'I love this product, it's the best thing I've ever bought!' -> Sentimiento Predicho: Positivo
Texto: 'This is a terrible experience, I'm so disappointed.' -> Sentimiento Predicho: Negativo
